In [7]:
import requests
import time
from bs4 import BeautifulSoup
import re
from tqdm import notebook
from collections import defaultdict

def get_plot(movie_url):
    response = requests.get("https://en.wikipedia.org"+movie_url)
    if response.status_code != 200:
        #print(response)
        return ""
    soup = BeautifulSoup(response.content, 'html.parser')
    plot = ""
    for tag in soup.find_all('h2'):
        if 'Plot' in tag.get_text() or 'Premise' in tag.get_text() or 'Synopsis' in tag.get_text() or 'Storyline' in tag.get_text():
            for sibling in tag.next_siblings:
                if sibling.name == 'p':
                    chk = re.sub('\[\d+\]', '', str(sibling.get_text()).strip())
                    plot += chk+ "\n"
                elif sibling.name == 'h2':
                    break
            break
    return plot

In [42]:
def get_british_movie_list(year):
    response = requests.get(f"https://en.wikipedia.org/wiki/List_of_British_films_of_{year}")
    if response.status_code != 200:
        print(country, year)
        return ""
    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.find_all("table")
    movie_list = []

    for tag in soup.find_all('h3'):
        if 'major releases' in tag.get_text().lower() or 'film premieres' in tag.get_text().lower():
            for sibling in tag.next_siblings:
                if sibling.name == 'table':
                    rows = sibling.find("tbody").find_all("tr")
                    for index, row in enumerate(rows):
                        for col in row.find_all('td'):
                            if not col.get_text().strip().isdigit():
                                if col.find('a', href=True):
                                    movie_list.append(col.find('a', href=True)['href'])
                                break
                elif sibling.name == 'h2':
                    break
    return movie_list

In [9]:
def check_month(title):
    if not title:
        return False
    for mon in ['Jan', 'Feb','Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']:
        if mon in title:
            return True
    return False

In [50]:
def get_movie_list(country, year):
    response = requests.get(f"https://en.wikipedia.org/wiki/List_of_{country}_films_of_{year}")
    if response.status_code != 200:
        print(country, year)
        
        return ""
    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.find_all("table")
    movie_list = []

    for tag in soup.find_all('h3'):
        if check_month(tag.get_text()):
            for sibling in tag.next_siblings:
                if sibling.name == 'table':
                    rows = sibling.find("tbody").find_all("tr")
                    for row in rows:
                        for col in row.find_all('td'):
                            if not col.get_text().strip().isdigit() and col.find('a', href=True):
                                movie_list.append(col.find('a', href=True)['href'])
                                break
                elif sibling.name == 'h2':
                    break
    return movie_list

In [52]:
movies_by_year = defaultdict(list)

In [53]:
for year in notebook.tqdm(range(2013, 2022)):
    movie_list = get_movie_list("Tamil", year)
    movies_by_year[year] = list()
    for movie in notebook.tqdm(movie_list, leave=False):
        movie_plot = get_plot(movie)
        if movie_plot!= '':
            movies_by_year[year].append(movie_plot)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/364 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/265 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/437 [00:00<?, ?it/s]

  0%|          | 0/242 [00:00<?, ?it/s]

  0%|          | 0/453 [00:00<?, ?it/s]

In [55]:
import json
with open("Tamil_plot_summaries_by_year.json", 'w') as f:
    json.dump(movies_by_year, f)